In [1]:
import albumentations #fast and flexible image augmentation library
import torch #provides a wide range of algorithms for deep learning
import joblib #provides lightweight pipelining/transparent disk-caching of functions and lazy re-evaluation (memoize pattern)
import torch.nn as nn #helps in creating and training of the neural network
import numpy as np #operations on arrays
import cv2 #read and preprocess images
import torch.nn.functional as F #applies a xD(ex. 2D) convolution over an input image composed of several input planes
import time #provides many ways of representing time in code
import import_ipynb #defines a “notebook loader” that allows you to ‘import’ other ipynb files into your current ipynb file
import CNN #the created model
import matplotlib.pyplot as plt #plotting library

importing Jupyter notebook from CNN.ipynb
Loading label binarizer...


In [2]:
aug = albumentations.Compose([
                albumentations.LongestMaxSize(224),
])

In [3]:
# load label binarizer
lb = joblib.load('D:\\Anaconda_zadachki\\RealTimeObjectDetection1\\Tensorflow\\workspace\\out\\lb.pkl')

In [4]:
# model to gpu
device = torch.device('cude' if torch.cuda.is_available() else 'cpu')

In [5]:
model = CNN.CustomCNN().to(device)
model.load_state_dict(torch.load('D:\\Anaconda_zadachki\\RealTimeObjectDetection1\\Tensorflow\\workspace\\out\\model.pth'))
print(model)
print('Model loaded')

CustomCNN(
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=128, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=58, bias=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
Model loaded


In [26]:
image = cv2.imread(f"D:\\Anaconda_zadachki\\RealTimeObjectDetection1\\Tensorflow\\workspace\\images\\test\\53.jpg")
image_copy = image.copy()
 
image = aug(image=np.array(image))['image']
image = np.transpose(image, (2, 0, 1)).astype(np.float32)
image = torch.tensor(image, dtype=torch.float).to(device)
image = image.unsqueeze(0)
print(image.shape)

torch.Size([1, 3, 224, 224])


In [27]:
start = time.time()
outputs = model(image)
_, preds = torch.max(outputs.data, 1)
print('PREDS', preds)
print(f"Predicted output: {lb.classes_[preds]}")
end = time.time()
print(f"{(end-start):.3f} seconds")
cv2.putText(image_copy, lb.classes_[preds], (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

PREDS tensor([49])
Predicted output: U
0.031 seconds


array([[[154, 156, 150],
        [156, 158, 152],
        [163, 165, 159],
        ...,
        [147, 155, 155],
        [128, 136, 136],
        [138, 146, 146]],

       [[155, 157, 151],
        [154, 156, 150],
        [152, 154, 148],
        ...,
        [131, 139, 139],
        [124, 132, 132],
        [151, 159, 159]],

       [[151, 152, 148],
        [157, 158, 154],
        [157, 158, 154],
        ...,
        [131, 139, 139],
        [138, 143, 144],
        [155, 160, 161]],

       ...,

       [[172, 162, 144],
        [190, 180, 162],
        [190, 179, 159],
        ...,
        [111, 109, 108],
        [132, 131, 127],
        [138, 137, 133]],

       [[175, 162, 146],
        [190, 178, 160],
        [170, 156, 137],
        ...,
        [137, 135, 134],
        [157, 156, 152],
        [137, 136, 132]],

       [[187, 174, 158],
        [177, 165, 147],
        [180, 166, 148],
        ...,
        [142, 140, 139],
        [153, 152, 148],
        [150, 149, 145]]

In [28]:
cv2.imwrite(f"D:\\Anaconda_zadachki\\RealTimeObjectDetection1\\Tensorflow\\workspace\\out\\53.jpg", image_copy)

True

In [29]:
from cv2 import __version__
__version__

'4.5.5'

In [30]:
cv2.imshow(f"D:\\Anaconda_zadachki\\RealTimeObjectDetection1\\Tensorflow\\workspace\\out\\53.jpg", image_copy)
cv2.waitKey(0)

-1